In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn import preprocessing

In [4]:
df = pd.read_csv("Huston College Lawsuit\Lawsuit.csv",sep=",")

In [5]:
df.head()

,Dept,Gender,Clin,Cert,Prate,Exper,Rank,Sal94,Sal95
0,1,1,0,0,7.4,9,3,77836,84612
1,1,1,0,0,6.7,10,2,69994,78497
2,1,1,0,0,8.1,6,1,62872,67756
3,1,1,1,1,5.1,27,3,155196,173220
4,1,1,0,0,7.0,10,3,89268,96099


In [7]:
df['hey'] = df['Sal95'] - df['Sal94']

In [10]:
df.describe()

,Dept,Gender,Clin,Cert,Prate,Exper,Rank,Sal94,Sal95,hey
count,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000
mean,3.651341,0.593870,0.613027,0.720307,4.932184,10.229885,1.896552,153593.344828,168906.655172,15313.310345
std,1.796655,0.492053,0.487993,0.449711,1.944430,6.227176,0.864262,80469.666720,88778.425411,8926.154373
min,1.000000,0.000000,0.000000,0.000000,1.300000,1.000000,1.000000,34514.000000,38675.000000,2312.000000
25%,2.000000,0.000000,0.000000,0.000000,3.200000,6.000000,1.000000,90771.000000,99972.000000,8494.000000
50%,4.000000,1.000000,1.000000,1.000000,4.400000,9.000000,2.000000,133284.000000,148117.000000,12955.000000
75%,5.000000,1.000000,1.000000,1.000000,6.900000,14.000000,3.000000,200543.000000,218955.000000,19364.000000
max,6.000000,1.000000,1.000000,1.000000,8.700000,37.000000,3.000000,428876.000000,472589.000000,53460.000000


In [4]:
from rpy2.robjects.packages import importr
utils = importr("utils")

In [37]:
import rpy2.ipython
%load_ext rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [38]:
%%R
library(dplyr)
library(ggplot2)
raw_data <- read.csv("./COMPASS Scores/compas-scores-two-years-violent.csv")
nrow(raw_data)

[1] 4743


In [39]:
%%R
df <- dplyr::select(raw_data, age, c_charge_degree, race, age_cat, score_text, sex, priors_count, 
                    days_b_screening_arrest, decile_score, is_recid, two_year_recid, c_jail_in, c_jail_out) %>% 
        filter(days_b_screening_arrest <= 30) %>%
        filter(days_b_screening_arrest >= -30) %>%
        filter(is_recid != -1) %>%
        filter(c_charge_degree != "O") %>%
        filter(score_text != 'N/A')
nrow(df)
write.table(df,"compas-scores-two-years-violent-filtered.csv",sep=";")


In [11]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [58]:
df = df.dropna()

In [5]:
df.shape

(1000, 5)

In [9]:
df.head()

,Age_Young,Age_Older,Gender,Expenditures_Low,Expenditures_High,Ethnicity
0,1,0,Female,1,0,White not Hispanic
1,0,1,Male,0,1,White not Hispanic
2,1,0,Male,1,0,Hispanic
3,0,1,Female,1,0,Hispanic
4,1,0,Male,1,0,White not Hispanic


In [34]:
df = df.drop(["Unnamed: 0","Employee Name"],axis=1)

In [76]:
df["sex"].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [7]:
df.describe()

,Age,Expenditures
count,1000.000000,1000.000000
mean,22.800000,18065.786000
std,18.462038,19542.830884
min,0.000000,222.000000
25%,12.000000,2898.750000
50%,18.000000,7026.000000
75%,26.000000,37712.750000
max,95.000000,75098.000000


In [51]:
import time

In [52]:
time.strftime('%H:%M:%S', time.gmtime(60))

'00:01:00'

In [10]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
categorical

['Gender', 'Ethnicity']

In [11]:
#Deep copy the original data
data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

Feature Gender
mapping {'Male': 1, 'Female': 0}
Feature Ethnicity
mapping {'Hispanic': 3, 'Asian': 1, 'Multi Race': 4, 'White not Hispanic': 7, 'Native Hawaiian': 5, 'American Indian': 0, 'Black': 2, 'Other': 6}


In [12]:
# Create new dataframe with dummy features
categorical_feats = df.select_dtypes(include=['object']).columns.tolist()
print(categorical_feats)

['Gender', 'Ethnicity']


In [13]:
df = pd.get_dummies(df, columns= categorical_feats)

In [14]:
df.head()

,Age_Young,Age_Older,Expenditures_Low,Expenditures_High,Gender_Female,Gender_Male,Ethnicity_American Indian,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_Multi Race,Ethnicity_Native Hawaiian,Ethnicity_Other,Ethnicity_White not Hispanic
0,1,0,1,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,1,0,1,0,0,0,0,0,0,0,1
2,1,0,1,0,0,1,0,0,0,1,0,0,0,0
3,0,1,1,0,1,0,0,0,0,1,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,0,0,0,1


In [15]:
df.to_csv("DDSData Binary.csv",index=None,sep=";")